In [2]:
import pandas as pd
import numpy as np
import requests
import json
import os
import time
import fred_msa
from datetime import date

pd.set_option('display.max_rows', 100)

In [3]:
csvs_to_ignore = ['new-series-to-use.csv']

In [10]:
filepath = '..\\fred-data-new'
data_files = set(os.listdir(filepath)).difference(set(csvs_to_ignore))

In [38]:
all_data = {}
for file in data_files:
    data = pd.read_csv(os.path.join(filepath, file))
    frequency = data.frequency[0]
    
    data.date = pd.to_datetime(data.date)
    data['year'] = [d.year for d in data.date]
    if frequency == 'Monthly':
        data['month'] = [d.month for d in data.date]
    all_data[file.replace('.csv', '')] = {'data' : data.drop(columns=['msa', 'title', 'id', 'frequency', 'seasonal_adjustment', 'date']), 'frequency' : data.frequency[0]}

In [39]:
dataset = all_data['Housing Inventory - Median Listing Price']['data'][['year', 'month', 'city', 'state', 'Housing Inventory: Median Listing Price']]

In [40]:
for k, v in all_data.items():
    if k != 'Housing Inventory - Median Listing Price':
        if v['frequency'] == 'Monthly':
            dataset = pd.merge(dataset, v['data'], how = 'left', on = ['year', 'month', 'city', 'state'], validate = 'one_to_one')
        else:
            dataset = pd.merge(dataset, v['data'], how = 'left', on = ['year', 'city', 'state'], validate = 'many_to_one')

In [41]:
obj_cols = ['city', 'state']

for col in dataset.columns:
    if col not in obj_cols:
        try:
            dataset[col] = dataset[col].astype('float')
        except:
            dataset = dataset.drop(columns=[col])

In [43]:
for col in dataset.columns:
    if 'Employees' in col:
        dataset[col] = dataset[col]/dataset['Resident Population']

In [44]:
dataset.to_csv(os.path.join(filepath, 'all-data.csv'), index=False)